# XGBoost Server Test with Iris Dataset


In [2]:
import os

import xgboost as xgb
from sklearn.datasets import load_iris

model_dir = "."
BST_FILE = "model.bst"

iris = load_iris()
y = iris["target"]
X = iris["data"]
dtrain = xgb.DMatrix(X, label=y)
param = {
    "max_depth": 6,
    "eta": 0.1,
    "silent": 1,
    "nthread": 4,
    "num_class": 10,
    "objective": "multi:softmax",
}
xgb_model = xgb.train(params=param, dtrain=dtrain)
model_file = os.path.join((model_dir), BST_FILE)
xgb_model.save_model(model_file)

Wrap model using s2i

## REST test

In [2]:
!cd .. && make build_rest

s2i build -E environment_rest ./xgboostserver seldonio/seldon-core-s2i-python37:0.11-SNAPSHOT seldonio/xgboostserver_rest:0.1
---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
Build completed successfully


In [3]:
!docker run --rm -d --name "xgboostserver"  -p 5000:5000 -e PREDICTIVE_UNIT_PARAMETERS='[{"type":"STRING","name":"model_uri","value":"file:///model"}]' -v ${PWD}:/model seldonio/xgboostserver_rest:0.3

5da13b570752a140ef7d0ab1c3dad07286bbb25dd266dd48b589d95296e7ab68


Send some random features that conform to the contract

In [1]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p

----------------------------------------
SENDING NEW REQUEST:

[[6.799 4.921 5.914 0.77 ]]
RECEIVED RESPONSE:
meta {
}
data {
  ndarray {
    values {
      number_value: 2.0
    }
  }
}




In [2]:
!docker rm xgboostserver --force

xgboostserver


## grpc test

In [6]:
!cd .. && make build_grpc

s2i build -E environment_grpc ./xgboostserver seldonio/seldon-core-s2i-python37:0.11-SNAPSHOT seldonio/xgboostserver_grpc:0.1
---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
Build completed successfully


In [5]:
!docker run --rm -d --name "xgboostserver"  -p 5000:5000 -e PREDICTIVE_UNIT_PARAMETERS='[{"type":"STRING","name":"model_uri","value":"file:///model"}]' -v ${PWD}:/model seldonio/xgboostserver_grpc:0.3

1dd48a0165687907227504460b996fb4ba64d1694c9f98c599686a3bf4e796aa


Test using NDArray payload

In [6]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p --grpc

----------------------------------------
SENDING NEW REQUEST:

[[5.136 2.385 7.006 0.359]]
RECEIVED RESPONSE:
meta {
}
data {
  ndarray {
    values {
      number_value: 2.0
    }
  }
}




Test using Tensor payload

In [7]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p --grpc --tensor

----------------------------------------
SENDING NEW REQUEST:

[[7.581 2.855 9.774 2.776]]
RECEIVED RESPONSE:
meta {
}
data {
  tensor {
    shape: 1
    values: 2.0
  }
}




In [8]:
!docker rm xgboostserver --force

xgboostserver
